## **Multi Query Retriever 개요**

**Multi Query Retriever**는 **Vector Store Retriever의 한계를 극복하기 위해 설계된 검색 방식**입니다.

- **단일 쿼리를 다양한 관점에서 변형하여 여러 개의 검색 쿼리를 생성**
- **생성된 모든 쿼리에 대해 문서를 검색하고, 중복된 결과를 제거하여 더 다양한 문서를 반환**
- LLM을 활용하여 자동으로 패러프레이징(Paraphrasing)된 여러 개의 쿼리를 생성


### 패러프레이징(Paraphrasing)이란?

**쉬운 예시:**

- 원본: "오늘 날씨가 좋다"
- 패러프레이징:
    - "오늘 하늘이 맑다"
    - "today 기분 좋은 날씨네"
    - "오늘은 화창하다"

모두 같은 뜻이지만 **다른 단어와 표현**을 사용한 거죠.


**Multi Query Retriever에서 패러프레이징이 중요한 이유:**

사용자가 "파이썬 배우는 방법"이라고 검색하면, AI가 자동으로 이를 여러 방식으로 바꿔서 검색합니다:

- "파이썬 학습 방법"
- "Python 공부하는 법"
- "파이썬 프로그래밍 입문"
- "파이썬 코딩 배우기"

- **장점**:
    - 사용자가 입력한 단일 쿼리를 **다양한 방식으로 변형하여 검색 범위 확장**
    - `"LangChain이란?"`이라는 질문에서 `"LangChain의 기능"`, `"LangChain의 개발 역사"` 등의 문서를 함께 검색하여 더 풍부한 결과 제공
- **단점**: 쿼리 수가 많아지면서 검색 시간이 증가할 수 있음

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 및 벡터스토어 설정
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embedding_model)

# LLM 모델 설정 (GPT 사용)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# Multi Query Retriever 생성
retriever = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(), llm=llm)

# 검색 실행
query = "LangChain 이란?"
retrieved_docs = retriever.get_relevant_documents(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)